# 0. Initialization On Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Colab Notebooks/21_BigCon'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}/datasets'.format(FOLDERNAME))

# 1. Call Library And Setting Working Directory

In [ ]:
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/21_BigCon/datasets')

# 2. Load Data

In [ ]:
unknown=pd.read_csv('unknown.csv')
unknown

In [ ]:
unknown=unknown[['base_date','em_g']]
unknown=unknown[20:32]
unknown.index=unknown.base_date
unknown=unknown[['em_g']]
unknown

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller_test = adfuller(unknown['em_g'], autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import itertools
def grid_search_pdq(a):
  '''
  - Input
  1. a(하나의 emd에 대한 하나의 feature로 나타나는 Series)
  2. d : decision_d를 바탕으로 얻어낸 d
  - Output : 최적의 ARIMA 모형을 바탕으로 한 7, 8월의 예측값
  '''
  # Grid Search
  p = d = q = range(0,3)
  pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
  combs = {} # stores aic and order pairs
  aics = [] # stores aics
  # Grid Search continued
  for combination in pdq:
    try:
      model = ARIMA(a, order=combination) # create all possible models
      model = model.fit()
      combs.update({model.aic : combination}) # store combinations
      aics.append(model.aic)
    except:
      continue

    best_aic = min(aics)

  # Model Creation and Forecasting
  model = ARIMA(a, order=combs[best_aic])
  model = model.fit()
  july,august = model.forecast(steps=2)[0] # 21년 7월, 8월 예측
  return (july, august)

In [ ]:
unknown_new=[grid_search_pdq(unknown)]

In [ ]:
unknown_new

In [ ]:
new=pd.DataFrame(unknown_new).T
new.index=[202107,202108]
new.columns=['em_g']
unknown_pred=pd.concat([unknown,new])
unknown_pred